In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Create Spark session
spark = SparkSession.builder.appName("InventoryCleaning").getOrCreate()

# Load raw CSV file
df = spark.read.csv("/content/inventory_log_raw.csv", header=True, inferSchema=True)

# Show original data
df.show(5)

#Remove duplicates
df_cleaned = df.dropDuplicates()

#Handle missing values - drop rows with nulls in important fields
df_cleaned = df_cleaned.dropna(subset=["item_name", "stock_left", "timestamp"])

#Calculate total sales per item (number of records per item_name)
sales_df = df_cleaned.groupBy("item_name").agg(count("*").alias("total_sales"))

# Show result
sales_df.show()

+----+---------+----------+----------------+--------+---------+
|  ID|item_name|stock_left|       timestamp|latitude|longitude|
+----+---------+----------+----------------+--------+---------+
| 712|   Item_C|       5.0|08-04-2025 12:29|25.83887| 62.51105|
|1121|   Item_C|       8.0|10-04-2025 13:31|21.09985| 44.42901|
|1186|   Item_C|       6.0|            NULL|18.07317| 39.43655|
|1768|   Item_C|      10.0|13-04-2025 19:17|18.94437| 52.70573|
|1828|   Item_C|       9.0|14-04-2025 03:06| 26.3342| 59.61011|
+----+---------+----------+----------------+--------+---------+
only showing top 5 rows

+---------+-----------+
|item_name|total_sales|
+---------+-----------+
|   Item_A|        403|
|   Item_D|        190|
|   Item_B|        217|
|   Item_E|        246|
|   Item_C|        470|
+---------+-----------+

